In [1]:
import cocotb
from cocotb.triggers import RisingEdge, Timer
from cocotb.clock import Clock
from cocotb.queue import Queue
import os
import subprocess
import random
import math
import pytest
from cocotb_test.simulator import run
from TBBase import TBBase
from ConstrainedRandom import ConstrainedRandom
from APB import APBMonitor, APBSlave, APBBase, APBCommandPacket

In [2]:
# Constrained random settings for command generation
DATA_WIDTH = 32
ADDR_WIDTH=32
STRB_WIDTH = 4
registers = 32 * STRB_WIDTH
clog2_registers = math.ceil(math.log2(registers))
pwrite_constraints = [(0, 0), (1, 1)]
pwrite_weights     = [1, 1]
min_high = (4  * STRB_WIDTH)-1
max_low  = (4  * STRB_WIDTH)
max_high = (32 * STRB_WIDTH)-1
paddr_constraints  = [(0, min_high), (max_low, max_high)]
paddr_weights      = [3, 1]
pprot_constraints  = [(0, 0), (1, 7)]
pprot_weights      = [4, 1]

In [3]:
cmd_packet = APBCommandPacket(
    data_width         = DATA_WIDTH,
    addr_width         = ADDR_WIDTH,
    strb_width         = STRB_WIDTH,
    pwrite_constraints = pwrite_constraints,
    pwrite_weights     = pwrite_weights,
    paddr_constraints  = paddr_constraints,
    paddr_weights      = paddr_weights,
    pprot_constraints  = pprot_constraints,
    pprot_weights      = pprot_weights
)

In [2]:
from cocotb_coverage.crv import Randomized
import random


In [6]:
class BusSignal(Randomized):
    def __init__(self, constraints):
        super().__init__()
        self.constraints = constraints
        self.key_list = list(sorted(self.constraints.keys()))
        
        # Initialize attributes
        self.i_valid = 0
        self.i_addr = 0
        self.i_data = 0
        self.i_cmd = 0
        self.i_ready = 0

        # Adding randomized signals with full ranges
        self.add_rand("i_valid", list(range(11)))
        self.add_rand("i_addr", list(range(256)))
        self.add_rand("i_data", list(range(256)))
        self.add_rand("i_cmd", [0, 3, 5, 9])
        self.add_rand("i_ready", [0, 1])

    def apply_constraints(self):
        for signal, (bins, weights) in self.constraints.items():
            choice = random.choices(bins, weights)[0]
            value = random.randint(choice[0], choice[1])
            setattr(self, signal, value)
    
    def set_constrained_random(self):
        self.randomize()
        self.apply_constraints()
        return {
            "i_valid": self.i_valid,
            "i_addr": self.i_addr,
            "i_data": self.i_data,
            "i_cmd": self.i_cmd,
            "i_ready": self.i_ready
        }

    def print(self):
        print('-'*120)
        for key in self.key_list:
            spaces = 20 - len(key)
            value = getattr(self, key)
            print(f' {key}:{" " * spaces}{value}')



In [10]:
# Define constraints and weights for the bus signals
bus_constraints = {
    "i_valid": ([(0, 0), (1, 1), (2, 10)], [10, 5, 1]),
    "i_addr": ([(0, 4), (16, 24), (32, 40), (48, 60)], [1, 2, 2, 1]),
    "i_data": ([(0, 63), (64, 127), (128, 191), (192, 255)], [1, 2, 2, 1]),
    "i_cmd": ([(0, 0), (1, 1), (2, 2), (3, 3)], [1, 1, 1, 1]),
    "i_ready": ([(0, 0), (1, 1)], [1, 1])
}

# Create the constrained random generator for the bus
bus_crand = BusSignal(bus_constraints)

In [11]:
for _ in range(10):  # Run for 100 clock cycles
    # Generate constrained random values for the bus
    bus_values = bus_crand.set_constrained_random()
    bus_crand.print()

------------------------------------------------------------------------------------------------------------------------
 i_addr:              38
 i_cmd:               2
 i_data:              21
 i_ready:             1
 i_valid:             0
------------------------------------------------------------------------------------------------------------------------
 i_addr:              34
 i_cmd:               1
 i_data:              189
 i_ready:             1
 i_valid:             0
------------------------------------------------------------------------------------------------------------------------
 i_addr:              17
 i_cmd:               3
 i_data:              153
 i_ready:             1
 i_valid:             0
------------------------------------------------------------------------------------------------------------------------
 i_addr:              40
 i_cmd:               0
 i_data:              133
 i_ready:             0
 i_valid:             0
-------------------------

In [88]:
from TBBase import TBBase
import cocotb
from cocotb.clock import Clock
from cocotb.triggers import RisingEdge, FallingEdge, ClockCycles, Timer
from cocotb.handle import SimHandleBase
from cocotb_bus.monitors import BusMonitor
from cocotb.utils import get_sim_time
from cocotb.queue import Queue
from cocotb.result import TestFailure
from cocotb_coverage.crv import Randomized
import os
import random
from ConstrainedRandom import ConstrainedRandom
from MemoryModel import MemoryModel
from dataclasses import dataclass
from collections import deque

# define the PWRITE mapping
pwrite = ['READ', 'WRITE']
apb_signals = [
    "PSEL",
    "PWRITE",
    "PENABLE",
    "PADDR",
    "PWDATA",
    "PRDATA",
    "PREADY"
]
apb_optional_signals = [
    "PPROT",
    "PSLVERR",
    "PSTRB"
]

@dataclass
class APBCycle:
    start_time: int
    count: int
    direction: str
    pwrite: int
    paddr: int
    pwdata: int
    pstrb: int
    prdata: int
    pprot: int
    pslverr: int

    def __eq__(self, other):
        if not isinstance(other, APBCycle):
            return NotImplemented
        if self.direction == 'WRITE':
            data_s = self.pwdata
            data_o = other.pwdata
        else:
            data_s = self.prdata
            data_o = other.prdata

        # Compare only specific fields
        return (self.direction == other.direction and
                self.paddr == other.paddr and
                data_s == data_o and
                self.pprot == other.pprot and
                self.pslverr == other.pslverr)


    def __str__(self):
        return  'APB Cycle\n'+\
                f"start_time: {self.start_time}\n"+\
                f"count:      {self.count}\n"+\
                f"direction:  {self.direction}\n"+\
                f"paddr:      0x{self.paddr:08X}\n"+\
                f"pwdata:     0x{self.pwdata:08X}\n"+\
                f"pstrb:      0x{self.pstrb:08b}\n" +\
                f"prdata:     0x{self.prdata:08X}\n"+\
                f"pprot:      0x{self.pprot:04X}\n"+\
                f"pslverr:    {self.pslverr}\n"

class APBTransaction(Randomized):
    def __init__(self, data_width, addr_width, strb_width,
                    constraints=None):
        super().__init__()
        self.start_time = 0
        self.data_width = data_width
        self.addr_width = addr_width
        self.strb_width = strb_width
        self.addr_mask  = (strb_width - 1)
        self.count = 0
        if constraints is None:
            addr_min_hi = (4  * self.STRB_WIDTH)-1
            addr_max_lo = (4  * self.STRB_WIDTH)
            addr_max_hi = (32 * self.STRB_WIDTH)-1
            self.constraints = {
                'last':   ([(0, 0), (1, 1)],
                            [1, 1]),
                'first':  ([(0, 0), (1, 1)],
                            [1, 1]),
                'pwrite': ([(0, 0), (1, 1)],
                            [1, 1]),
                'paddr':  ([(0, addr_min_hi), (addr_max_lo, addr_max_hi)],
                            [4, 1]),
                'pprot':  ([(0, 0), (1, 7)],
                            [4, 1])
            }
        else:
            self.constraints = constraints

        self.last = 0
        self.first = 0
        self.pwrite = 0
        self.paddr = 0
        self.pprot = 0
        self.cycle = APBCycle(
            start_time=0,
            count=0,
            direction='unknown',
            pwrite=0,
            paddr=0,
            pwdata=0,
            prdata=0,
            pstrb=0,
            pprot=0,
            pslverr=0
        )

        # Adding randomized signals with full ranges
        self.add_rand("last",   [0, 1])
        self.add_rand("first",  [0, 1])
        self.add_rand("pwrite", [0, 1])
        self.add_rand("paddr",  list(range(2**10)))
        self.add_rand("pprot",  list(range(8)))


    def apply_constraints(self):
        for signal, (bins, weights) in self.constraints.items():
            choice = random.choices(bins, weights)[0]
            value = random.randint(choice[0], choice[1])
            setattr(self, signal, value)


    def set_constrained_random(self):
        self.randomize()
        self.apply_constraints()
        self.cycle.paddr     = self.paddr & ~self.addr_mask
        self.cycle.direction = pwrite[self.pwrite]
        self.cycle.pwrite    = self.pwrite
        self.cycle.pwdata    = random.randint(0, (1 << self.data_width) - 1)
        self.cycle.pstrb     = random.randint(0, (1 << self.strb_width) - 1)
        return self.cycle


    def pack_cmd_packet(self):
        """
        Pack the command packet into a single integer.
        """
        return (
            (self.last         << (self.addr_width + self.data_width + self.strb_width + 5)) |
            (self.first        << (self.addr_width + self.data_width + self.strb_width + 4)) |
            (self.cycle.pwrite << (self.addr_width + self.data_width + self.strb_width + 3)) |
            (self.cycle.pprot  << (self.addr_width + self.data_width + self.strb_width)) |
            (self.cycle.pstrb  << (self.addr_width + self.data_width)) |
            (self.cycle.paddr  << self.data_width) |
            self.cycle.pwdata
        )


    def unpack_cmd_packet(self, packed_packet):
        """
        Unpack a packed command packet into its components.
        """
        self.last         = (packed_packet >> (self.addr_width + self.data_width + self.strb_width + 5)) & 0x1
        self.first        = (packed_packet >> (self.addr_width + self.data_width + self.strb_width + 4)) & 0x1
        self.cycle.pwrite = (packed_packet >> (self.addr_width + self.data_width + self.strb_width + 3)) & 0x1
        self.cycle.pprot  = (packed_packet >> (self.addr_width + self.data_width + self.strb_width)) & 0x7
        self.cycle.pstrb  = (packed_packet >> (self.addr_width + self.data_width)) & ((1 << self.strb_width) - 1)
        self.cycle.paddr  = (packed_packet >> self.data_width) & ((1 << self.addr_width) - 1)
        self.cycle.pwdata = packed_packet & ((1 << self.data_width) - 1)


    def __str__(self):
        """
        Return a string representation of the command packet for debugging.
        """
        return (f'{self.cycle}')


In [89]:
DATA_WIDTH = 32
STRB_WIDTH = DATA_WIDTH // 8
ADDR_WIDTH = 9

addr_min_hi = (4  * STRB_WIDTH)-1
addr_max_lo = (4  * STRB_WIDTH)
addr_max_hi = (32 * STRB_WIDTH)-1
apb_cmd_constraints = {
'last':   ([(0, 0), (1, 1)],
                [1, 1]),
'first':  ([(0, 0), (1, 1)],
                [1, 1]),
'pwrite': ([(0, 0), (1, 1)],
                [1, 1]),
'paddr':  ([(0, addr_min_hi), (addr_max_lo, addr_max_hi)],
                [4, 1]),
'pprot':  ([(0, 0), (1, 7)],
                [4, 1])
        }


cmd_packet = APBTransaction(
        data_width         = DATA_WIDTH,
        addr_width         = ADDR_WIDTH,
        strb_width         = STRB_WIDTH,
        constraints        = apb_cmd_constraints
        )

In [101]:
transaction = cmd_packet.set_constrained_random()
print(transaction)


APB Cycle
start_time: 0
count:      0
direction:  WRITE
paddr:      0x00000000
pwdata:     0xBF0DBE63
pstrb:      0x00000010
prdata:     0x00000000
pprot:      0x0000
pslverr:    0

